In [1]:
import io

import pandas as pd
from Bio import Entrez, SeqIO
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

# Укажите ваш email (обязательно для использования Entrez)
Entrez.email = "your.email@example.com"

In [2]:
with open("db_curated_server_info.txt", "r") as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()
    if line and not line.startswith("#"):
        key, value = line.split("=", 1)
        config[key] = value.strip()

server_name = config.get("server_name")
srever_port = int(config.get("srever_port"))
ssh_password = config.get("ssh_password")
ssh_username = config.get("ssh_username")
db_adress = config.get("db_adress")
db_port = int(config.get("db_port"))

In [3]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

46107


In [4]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [5]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [6]:
add_histone = (
    "INSERT INTO histone "
    "(id, level, taxonomic_span, taxonomic_span_id, description, parent) "
    "VALUES (%(id)s, %(level)s, %(taxonomic_span)s, %(taxonomic_span_id)s, %(description)s, %(parent)s)"
)
add_histone_description = (
    "INSERT INTO histone_description "
    "(summary, taxonomy, genes, evolution, expression, knock_out, function, sequence, localization, deposition, structure, interactions, disease, caveats) "
    "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
)
# add_publication = (
#     "INSERT INTO publication "
#     "(id, title, doi, author, year) "
#     "VALUES (%(id)s, %(title)s, %(doi)s, %(author)s, %(year)s)"
# )
# add_sequence = (
#     "INSERT INTO sequence "
#     "(accession, variant, gi, ncbi_gene_id, hgnc_gene_name, taxonomy_id, organism, phylum, class, taxonomy_group, info, sequence, variant_under_consideration) "
#     "VALUES (%(accession)s, %(variant)s, %(gi)s, %(ncbi_gene_id)s, %(hgnc_gene_name)s, %(taxonomy_id)s, %(organism)s, %(phylum)s, %(class)s, %(taxonomy_group)s, %(info)s, %(sequence)s, %(variant_under_consideration)s)"
# )
# add_sequence_has_publication = (
#     "INSERT INTO sequence_has_publication "
#     "(sequence_accession, publication_id) "
#     "VALUES (%s, %s)"
# )
add_alternate_names = (
    "INSERT INTO alternative_name "
    "(name, taxonomy, gene, splice, histone) "
    "VALUES (%(name)s, %(taxonomy)s, %(gene)s, %(splice)s, %(histone)s)"
)
# add_histone_has_publication = (
#     "INSERT INTO histone_has_publication "
#     "(histone_id, publication_id) "
#     "VALUES (%s, %s)"
# )

In [7]:
data_histone_description = {
    "summary": None,
    "taxonomy": None,
    "genes": None,
    "evolution": None,
    "expression": None,
    "knock_out": None,
    "function": None,
    "sequence": None,
    "localization": None,
    "deposition": None,
    "structure": None,
    "interactions": None,
    "disease": None,
    "caveats": None,
}

# To Do H4

## <span style="color:green">Add a new node H4.G_(Hominidae) to H4</span>

## <span style="color:green">Add description to H4.G_(Hominidae)</span>

### <span style="color:green">Add summary</span>

```H4.G_(Hominidae) is a hominid-specific histone H4 variant, also known as H4G or H4.7, which promotes breast cancer proliferation by destabilizing nucleolar chromatin [pang_histone_2020, talbert_histone_2021, lai_roles_2024]. H4.G is encode by clustered gene H4C7. Since the expression of this gene has been described only in tumor cells and its product, H4.G, is incapable of forming stable nucleosomes in vitro, there is no compelling evidence that H4.G is a bona fide histone variant in normal cells [seal_standardized_2022].```

### <span style="color:green">Add gene</span>

```H4.G is encode by clustered gene H4C7.```

### <span style="color:green">Add sequence</span>

```H4.G shares 85% identity with canonical histone H4, differing by 15 amino acid residues located predominantly near the N-terminal α1, α2, and α3 regions, and possessing a C-terminus shortened by five residues [long_novel_2019].```

### <span style="color:green">Add expression</span>

```Although H4.G expression is observed at low levels in various human cell lines, its significant upregulation is found primarily in breast cancer, as well as in T-cell prolymphocytic leukemia, HuR-silenced thyroid carcinoma, and endometrioid carcinoma [pang_histone_2020, lai_roles_2024].```

### <span style="color:green">Add localization</span>

```It is predominantly localized to the nucleolus which contains the nucleolar chromatin [lai_roles_2024].```

### <span style="color:green">Add function</span>

```The functional role of H4.G in cells is likely to loosen the structure of nucleolar chromatin by forming destabilized nucleosome-like structures, thereby enhancing ribosomal RNA (rRNA) transcription and ribosome biogenesis, which promotes tumor cell growth [long_novel_2019, lai_roles_2024].```

### <span style="color:green">Add deposition</span>

```H4.G is not stably incorporated into nucleolar chromatin even with the assistance of the chaperone NPM1; instead, the H4.G-NPM1 complex forms unstable nucleosome-like structures that rapidly dissociate, and this very property appears to facilitate chromatin loosening [pang_histone_2020].```

### <span style="color:green">Add knock_out</span>

```H4.G knockdown in MCF7 breast cancer cells significantly impairs cell growth, proliferation, rRNA synthesis, and protein synthesis [long_novel_2019, pang_histone_2020].```

### <span style="color:green">Add structure</span>

```H4.G is unable to form stable nucleosomes in vitro, even in the presence of the chaperone NPM1; instead, it likely forms transient nucleosome-like structures that rapidly dissociate [pang_histone_2020].```

### <span style="color:green">Add disease</span>

```H4.G is overexpressed in a number of cancers, and its expression level positively correlates with cancer stage progression, as demonstrated in breast cancer, where it promotes tumor growth by facilitating rRNA and protein synthesis [lai_roles_2024].```

## <span style="color:green">Update summary of H4</span>

### <span style="color:green">Update summary</span>

```H4 is one of the core histones. Two H4s and two H3s form H3-H4 tetramer via "hand shake" and "four helix bundle" motifs. Two H2A-H2B dimers then associate with H3-H4 tetramer to form complete nucleosome core. H4 is the most conservative histone type and has very few known variants. Structure of H4 has a histone fold domain and a flexible N-terminal tail. H4 provides sites for H2B interaction via "four-helix bundle" and forms a small β-sheet with H2Aa. Acetylation of the H4 tail within the basic patch region (K5, K8, K12, K16, K20) modulates its conformational ensemble through electrostatic effects, alterations in contact patterns, and subtle changes in the protonation of key residues, thereby contributing to the regulation of chromatin architecture [dewing_acetylation-dependent_2024].```

## <span style="color:green">Add description to cH4_(Vertebrata)</span>

### <span style="color:green">Add summary</span>

```cH4_(Vertebrata) clustered H4 histones in vertebrates, often called canonical, replication-dependent, replication-coupled or "bulk" H4 histones. This is a subclass of cH4_(Animals) (see it for a detailed description). Asymmetric dimethylation of histone H4 at arginine 3 (H4R3me2a), catalyzed by PRMT1 in vertebrates, constitutes a key epigenetic mark for the establishment and maintenance of open and transcriptionally active chromatin at regulatory regions of the vertebrate genome, particularly at enhancers and super-enhancers [sudhakar_histone_2024].```

# DONE

## <span style="color:black">Add a new node H4.G_(Hominidae) to H4</span>

In [8]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H4.G_(Hominidae)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations


In [9]:
data_histone = {
    "id": "H4.G_(Hominidae)",
    "level": "variant_group",
    "taxonomic_span": "Hominidae",
    "taxonomic_span_id": "9604",
    "description": None,
    "parent": "H4",
}
cursor.execute(add_histone, data_histone)

In [10]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H4.G_(Hominidae)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H4.G_(Hominidae),variant_group,Hominidae,9604,None,H4,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [11]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Add description to H4.G_(Hominidae)</span>

### <span style="color:black">Add summary</span>

```H4.G_(Hominidae) is a hominid-specific histone H4 variant, also known as H4G or H4.7, which promotes breast cancer proliferation by destabilizing nucleolar chromatin [pang_histone_2020, talbert_histone_2021, lai_roles_2024]. H4.G is encode by clustered gene H4C7. Since the expression of this gene has been described only in tumor cells and its product, H4.G, is incapable of forming stable nucleosomes in vitro, there is no compelling evidence that H4.G is a bona fide histone variant in normal cells [seal_standardized_2022].```

### <span style="color:black">Add gene</span>

```H4.G is encode by clustered gene H4C7.```

### <span style="color:black">Add sequence</span>

```H4.G shares 85% identity with canonical histone H4, differing by 15 amino acid residues located predominantly near the N-terminal α1, α2, and α3 regions, and possessing a C-terminus shortened by five residues [long_novel_2019].```

### <span style="color:black">Add expression</span>

```Although H4.G expression is observed at low levels in various human cell lines, its significant upregulation is found primarily in breast cancer, as well as in T-cell prolymphocytic leukemia, HuR-silenced thyroid carcinoma, and endometrioid carcinoma [pang_histone_2020, lai_roles_2024].```

### <span style="color:black">Add localization</span>

```It is predominantly localized to the nucleolus which contains the nucleolar chromatin [lai_roles_2024].```

### <span style="color:black">Add function</span>

```The functional role of H4.G in cells is likely to loosen the structure of nucleolar chromatin by forming destabilized nucleosome-like structures, thereby enhancing ribosomal RNA (rRNA) transcription and ribosome biogenesis, which promotes tumor cell growth [long_novel_2019, lai_roles_2024].```

### <span style="color:black">Add deposition</span>

```H4.G is not stably incorporated into nucleolar chromatin even with the assistance of the chaperone NPM1; instead, the H4.G-NPM1 complex forms unstable nucleosome-like structures that rapidly dissociate, and this very property appears to facilitate chromatin loosening [pang_histone_2020].```

### <span style="color:black">Add knock_out</span>

```H4.G knockdown in MCF7 breast cancer cells significantly impairs cell growth, proliferation, rRNA synthesis, and protein synthesis [long_novel_2019, pang_histone_2020].```

### <span style="color:black">Add structure</span>

```H4.G is unable to form stable nucleosomes in vitro, even in the presence of the chaperone NPM1; instead, it likely forms transient nucleosome-like structures that rapidly dissociate [pang_histone_2020].```

### <span style="color:black">Add disease</span>

```H4.G is overexpressed in a number of cancers, and its expression level positively correlates with cancer stage progression, as demonstrated in breast cancer, where it promotes tumor growth by facilitating rRNA and protein synthesis [lai_roles_2024].```

In [12]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H4.G_(Hominidae)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H4.G_(Hominidae),variant_group,Hominidae,9604,None,H4,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [13]:
data_histone_description = {
    "summary": "H4.G_(Hominidae) is a hominid-specific histone H4 variant, also known as H4G or H4.7, which promotes breast cancer proliferation by destabilizing nucleolar chromatin [pang_histone_2020, talbert_histone_2021, lai_roles_2024]. H4.G is encode by clustered gene H4C7. Since the expression of this gene has been described only in tumor cells and its product, H4.G, is incapable of forming stable nucleosomes in vitro, there is no compelling evidence that H4.G is a bona fide histone variant in normal cells [seal_standardized_2022].",
    "taxonomy": None,
    "genes": "H4.G is encode by clustered gene H4C7.",
    "evolution": None,
    "expression": "Although H4.G expression is observed at low levels in various human cell lines, its significant upregulation is found primarily in breast cancer, as well as in T-cell prolymphocytic leukemia, HuR-silenced thyroid carcinoma, and endometrioid carcinoma [pang_histone_2020, lai_roles_2024].",
    "knock_out": "H4.G knockdown in MCF7 breast cancer cells significantly impairs cell growth, proliferation, rRNA synthesis, and protein synthesis [long_novel_2019, pang_histone_2020].",
    "function": "The functional role of H4.G in cells is likely to loosen the structure of nucleolar chromatin by forming destabilized nucleosome-like structures, thereby enhancing ribosomal RNA (rRNA) transcription and ribosome biogenesis, which promotes tumor cell growth [long_novel_2019, lai_roles_2024].",
    "sequence": "H4.G shares 85% identity with canonical histone H4, differing by 15 amino acid residues located predominantly near the N-terminal α1, α2, and α3 regions, and possessing a C-terminus shortened by five residues [long_novel_2019].",
    "localization": "It is predominantly localized to the nucleolus which contains the nucleolar chromatin [lai_roles_2024].",
    "deposition": "H4.G is not stably incorporated into nucleolar chromatin even with the assistance of the chaperone NPM1; instead, the H4.G-NPM1 complex forms unstable nucleosome-like structures that rapidly dissociate, and this very property appears to facilitate chromatin loosening [pang_histone_2020].",
    "structure": "H4.G is unable to form stable nucleosomes in vitro, even in the presence of the chaperone NPM1; instead, it likely forms transient nucleosome-like structures that rapidly dissociate [pang_histone_2020].",
    "interactions": None,
    "disease": "H4.G is overexpressed in a number of cancers, and its expression level positively correlates with cancer stage progression, as demonstrated in breast cancer, where it promotes tumor growth by facilitating rRNA and protein synthesis [lai_roles_2024].",
    "caveats": None,
}
cursor.execute(add_histone_description, tuple(data_histone_description.values()))

histone_description_id = cursor.lastrowid
query = f"UPDATE histone SET description={histone_description_id} WHERE id='H4.G_(Hominidae)'"
cursor.execute(query)

In [14]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H4.G_(Hominidae)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H4.G_(Hominidae),variant_group,Hominidae,9604,258,H4,258,H4.G_(Hominidae) is a hominid-specific histone...,None,H4.G is encode by clustered gene H4C7.,...,H4.G knockdown in MCF7 breast cancer cells sig...,The functional role of H4.G in cells is likely...,H4.G shares 85% identity with canonical histon...,It is predominantly localized to the nucleolus...,H4.G is not stably incorporated into nucleolar...,H4.G is unable to form stable nucleosomes in v...,None,"H4.G is overexpressed in a number of cancers, ...",None,None


In [15]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Add a alternate name H4.7 to H4.G_(Hominidae)</span>

In [18]:
query = "SELECT * FROM alternative_name " "WHERE histone='H4.G_(Hominidae)'"
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone


In [19]:
data_alternate_name = {
    "name": "H4.7",
    "taxonomy": None,
    "gene": None,
    "splice": None,
    "histone": "H4.G_(Hominidae)",
}
cursor.execute(add_alternate_names, data_alternate_name)

In [20]:
query = "SELECT * FROM alternative_name " "WHERE histone='H4.G_(Hominidae)'"
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,122,H4.7,None,None,None,H4.G_(Hominidae)


In [21]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Update summary of H4</span>

### <span style="color:black">Update summary</span>

```H4 is one of the core histones. Two H4s and two H3s form H3-H4 tetramer via "hand shake" and "four helix bundle" motifs. Two H2A-H2B dimers then associate with H3-H4 tetramer to form complete nucleosome core. H4 is the most conservative histone type and has very few known variants. Structure of H4 has a histone fold domain and a flexible N-terminal tail. H4 provides sites for H2B interaction via "four-helix bundle" and forms a small β-sheet with H2Aa. Acetylation of the H4 tail within the basic patch region (K5, K8, K12, K16, K20) modulates its conformational ensemble through electrostatic effects, alterations in contact patterns, and subtle changes in the protonation of key residues, thereby contributing to the regulation of chromatin architecture [dewing_acetylation-dependent_2024].```

In [22]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H4'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H4,type,null,null,6,None,6,H4 is one of the core histones. Two H4s and tw...,null,null,...,null,null,null,null,null,null,null,null,null,None


In [23]:
summary_desc = 'H4 is one of the core histones. Two H4s and two H3s form H3-H4 tetramer via "hand shake" and "four helix bundle" motifs. Two H2A-H2B dimers then associate with H3-H4 tetramer to form complete nucleosome core. H4 is the most conservative histone type and has very few known variants. Structure of H4 has a histone fold domain and a flexible N-terminal tail. H4 provides sites for H2B interaction via "four-helix bundle" and forms a small β-sheet with H2Aa. Acetylation of the H4 tail within the basic patch region (K5, K8, K12, K16, K20) modulates its conformational ensemble through electrostatic effects, alterations in contact patterns, and subtle changes in the protonation of key residues, thereby contributing to the regulation of chromatin architecture [dewing_acetylation-dependent_2024].'
query = f"UPDATE histone_description SET summary='{summary_desc}' WHERE id=6"
print(query)
cursor.execute(query)

UPDATE histone_description SET summary='H4 is one of the core histones. Two H4s and two H3s form H3-H4 tetramer via "hand shake" and "four helix bundle" motifs. Two H2A-H2B dimers then associate with H3-H4 tetramer to form complete nucleosome core. H4 is the most conservative histone type and has very few known variants. Structure of H4 has a histone fold domain and a flexible N-terminal tail. H4 provides sites for H2B interaction via "four-helix bundle" and forms a small β-sheet with H2Aa. Acetylation of the H4 tail within the basic patch region (K5, K8, K12, K16, K20) modulates its conformational ensemble through electrostatic effects, alterations in contact patterns, and subtle changes in the protonation of key residues, thereby contributing to the regulation of chromatin architecture [dewing_acetylation-dependent_2024].' WHERE id=6


In [24]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H4'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df["summary"].values[0]

'H4 is one of the core histones. Two H4s and two H3s form H3-H4 tetramer via "hand shake" and "four helix bundle" motifs. Two H2A-H2B dimers then associate with H3-H4 tetramer to form complete nucleosome core. H4 is the most conservative histone type and has very few known variants. Structure of H4 has a histone fold domain and a flexible N-terminal tail. H4 provides sites for H2B interaction via "four-helix bundle" and forms a small β-sheet with H2Aa. Acetylation of the H4 tail within the basic patch region (K5, K8, K12, K16, K20) modulates its conformational ensemble through electrostatic effects, alterations in contact patterns, and subtle changes in the protonation of key residues, thereby contributing to the regulation of chromatin architecture [dewing_acetylation-dependent_2024].'

In [25]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Add description to cH4_(Vertebrata)</span>

### <span style="color:black">Add summary</span>

```cH4_(Vertebrata) clustered H4 histones in vertebrates, often called canonical, replication-dependent, replication-coupled or "bulk" H4 histones. This is a subclass of cH4_(Animals) (see it for a detailed description). Asymmetric dimethylation of histone H4 at arginine 3 (H4R3me2a), catalyzed by PRMT1 in vertebrates, constitutes a key epigenetic mark for the establishment and maintenance of open and transcriptionally active chromatin at regulatory regions of the vertebrate genome, particularly at enhancers and super-enhancers [sudhakar_histone_2024].```

In [26]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cH4_(Vertebrata)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cH4_(Vertebrata),variant,Vertebrata,7742,None,cH4_(Animals),None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [27]:
data_histone_description = {
    "summary": 'cH4_(Vertebrata) clustered H4 histones in vertebrates, often called canonical, replication-dependent, replication-coupled or "bulk" H4 histones. This is a subclass of cH4_(Animals) (see it for a detailed description). Asymmetric dimethylation of histone H4 at arginine 3 (H4R3me2a), catalyzed by PRMT1 in vertebrates, constitutes a key epigenetic mark for the establishment and maintenance of open and transcriptionally active chromatin at regulatory regions of the vertebrate genome, particularly at enhancers and super-enhancers [sudhakar_histone_2024].',
    "taxonomy": None,
    "genes": None,
    "evolution": None,
    "expression": None,
    "knock_out": None,
    "function": None,
    "sequence": None,
    "localization": None,
    "deposition": None,
    "structure": None,
    "interactions": None,
    "disease": None,
    "caveats": None,
}
cursor.execute(add_histone_description, tuple(data_histone_description.values()))

histone_description_id = cursor.lastrowid
query = f"UPDATE histone SET description={histone_description_id} WHERE id='cH4_(Vertebrata)'"
cursor.execute(query)

In [28]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cH4_(Vertebrata)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df["summary"].values[0]

'cH4_(Vertebrata) clustered H4 histones in vertebrates, often called canonical, replication-dependent, replication-coupled or "bulk" H4 histones. This is a subclass of cH4_(Animals) (see it for a detailed description). Asymmetric dimethylation of histone H4 at arginine 3 (H4R3me2a), catalyzed by PRMT1 in vertebrates, constitutes a key epigenetic mark for the establishment and maintenance of open and transcriptionally active chromatin at regulatory regions of the vertebrate genome, particularly at enhancers and super-enhancers [sudhakar_histone_2024].'

In [29]:
# Make sure data is committed to the database
conn.commit()

# Close connections

In [30]:
cursor.close()
conn.close()
tunnel.stop()